In [22]:
import glob
import pandas as pd
import lightgbm as lgb
from IPython.display import display
from sklearn.model_selection import cross_val_score

In [38]:
path_in_trains = "../input_data/train/*.csv"
path_in_test = "../input_data/test.csv"

In [2]:
dfs = []
for fpath in sorted(glob.glob(path_in_trains)):
    _df = pd.read_csv(fpath, index_col=0)
    dfs.append(_df)
df_train = pd.concat(dfs)
df_test = pd.read_csv(path_in_test, index_col=0)

/Users/nsmt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_train.head(2)

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
1060685,中古マンション等,NaN,1108,北海道,札幌市厚別区,大谷地東,大谷地,8,３ＬＤＫ,80,...,NaN,NaN,NaN,準工業地域,60.0,200.0,2009年第４四半期,未改装,NaN,7.079181
1005580,中古マンション等,NaN,1101,北海道,札幌市中央区,南９条西,中島公園,5,１ＤＫ,30,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2018年第３四半期,未改装,NaN,6.755875


In [4]:
df_test.head(2)

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
ID,,,,,,,,,,,,,,,,,,,,,
1000000,中古マンション等,NaN,1101,北海道,札幌市中央区,旭ケ丘,円山公園,26,３ＬＤＫ,75,...,住宅,NaN,NaN,NaN,第１種低層住居専用地域,40.0,60.0,2020年第２四半期,未改装,NaN
1000056,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,1,２ＬＤＫ,55,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2020年第１四半期,未改装,NaN


## # 各列のチェック

とりあえず、各列の特徴をざっくり見ていきます。

ここでは、3つの値を見てみましょう。

 - 列の型
 - NaNの割合
 - 値の種類


In [14]:
# ここもうちょっといい書き方があってもいいと思うのだが…
dtypes = []
val_counts_train = []
val_counts_test = []
isnull_train = []
isnull_test = []
isnull_train_series = 100 * df_train.isnull().sum() / len(df_train)
isnull_test_series = 100 * df_test.isnull().sum() / len(df_test)
for col in df_test.columns:
    dtypes.append(str(df_train[col].dtype))
    val_counts_train.append(len(df_train[col].value_counts()))
    val_counts_test.append(len(df_test[col].value_counts()))
    isnull_train.append(isnull_train_series[col])
    isnull_test.append(isnull_test_series[col])

In [25]:
inds = ["型", "値の種類_train", "値の種類_test", "NaN率_train", "NaN率_test"]
df_eda = pd.DataFrame([dtypes, val_counts_train, val_counts_test, isnull_train, isnull_test], columns=df_test.columns, index=inds).T
df_eda.query("型 == 'object' and 値の種類_train > 1")

,型,値の種類_train,値の種類_test,NaN率_train,NaN率_test
都道府県名,object,47,47,0,0
市区町村名,object,618,526,0,0
地区名,object,12046,5515,0.0456577,0.0154115
最寄駅：名称,object,3831,2550,0.41092,0.0667831
最寄駅：距離（分）,object,34,34,3.61575,0.0667831
間取り,object,63,27,3.41131,4.73646
面積（㎡）,object,195,48,0,0
建築年,object,76,59,2.8609,3.4008
建物の構造,object,12,8,2.1853,6.49851
用途,object,22,10,7.23887,30.7511


まず、値の種類が1以下のものは除外します。（df_edaの中身を見てみるとわかりますが、値が全く入っていない列などが結構あります。。）

また、取引時点はtrainと比較して、testは直近2四半期のデータであることがわかります。

その他列に関しては、trainデータの中身なども鑑み、以下のように決めました。

## # 前処理方針

| 変数名       | 対応                               |
|-----------|----------------------------------|
| 都道府県名     | 47数値に変換                          |
| 市区町村名     | 市区町村コードと同じであった…削除                |
| 地区名       | カテゴリカルとして扱う                      |
| 最寄駅：名称    | カテゴリカルとして扱う                      |
| 最寄駅：距離（分） | ほぼfloat（◯◯〜◯◯分と記載があるものをfloatに変換） |
| 間取り       | 特殊処理（部屋数、L、D、K、Sの数を抽出）           |
| 面積（㎡）     | ほぼint（2000m2以上と記載があるものを2000に変換）  |
| 建築年       | 特殊処理（西暦に変換）                      |
| 建物の構造     | カテゴリカルとして扱う                      |
| 用途        | カテゴリカルとして扱う                      |
| 今後の利用目的   | カテゴリカルとして扱う                      |
| 都市計画      | カテゴリカルとして扱う                      |
| 取引時点      | 特殊処理（年と四半期を数値に変換）                |
| 改装        | 01に変換                            |
| 取引の事情等    | NaN率がかなり高いため削除                   |

In [26]:
# 値の種類が1種類しかないものを除外
dropcols = ["取引の事情等", "市区町村名"]
for col in df_train.columns:
    len_valuecounts = len(df_train[col].value_counts())
    if len_valuecounts <= 1:
        dropcols.append(col)
df_train.drop(dropcols, axis=1, inplace=True)
df_test.drop(dropcols, axis=1, inplace=True)

In [27]:
df_test.head(1)

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
ID,,,,,,,,,,,,,,,,
1000000,1101,北海道,旭ケ丘,円山公園,26,３ＬＤＫ,75,昭和64年,ＲＣ,NaN,住宅,第１種低層住居専用地域,40.0,60.0,2020年第２四半期,未改装


In [28]:
df_train.head(1)

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,
1060685,1108,北海道,大谷地東,大谷地,8,３ＬＤＫ,80,平成7年,ＳＲＣ,住宅,NaN,準工業地域,60.0,200.0,2009年第４四半期,未改装,7.079181


In [31]:
dd = {
    "北海道": 1, 
    "青森県": 2, 
    "岩手県": 3, 
    "宮城県": 4, 
    "秋田県": 5, 
    "山形県": 6, 
    "福島県": 7, 
    "茨城県": 8, 
    "栃木県": 9, 
    "群馬県": 10, 
    "埼玉県": 11, 
    "千葉県": 12, 
    "東京都": 13, 
    "神奈川県": 14, 
    "新潟県": 15, 
    "富山県": 16, 
    "石川県": 17, 
    "福井県": 18, 
    "山梨県": 19, 
    "長野県": 20, 
    "岐阜県": 21, 
    "静岡県": 22, 
    "愛知県": 23, 
    "三重県": 24, 
    "滋賀県": 25, 
    "京都府": 26, 
    "大阪府": 27, 
    "兵庫県": 28, 
    "奈良県": 29, 
    "和歌山県": 30, 
    "鳥取県": 31, 
    "島根県": 32, 
    "岡山県": 33, 
    "広島県": 34, 
    "山口県": 35, 
    "徳島県": 36, 
    "香川県": 37, 
    "愛媛県": 38, 
    "高知県": 39, 
    "福岡県": 40, 
    "佐賀県": 41, 
    "長崎県": 42, 
    "熊本県": 43, 
    "大分県": 44, 
    "宮崎県": 45, 
    "鹿児島県": 46, 
    "沖縄県": 47
}

def preprocess(df, flag):
    # 都道府県名
    df["都道府県名"] = df["都道府県名"].replace(dd)
    
    # 取引時点
    replaced = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99",
    }
    d = {}
    for val in df["取引時点"].value_counts().keys():
        new_float = 0.0
        for k, v in replaced.items():
            if k in val:
                new_float = float(val.replace(k, v))
        d[val] = new_float
    df["取引時点"] = df["取引時点"].replace(d)

    # 改装
    col = "改装"
    df[col] = df[col].replace({"未改装": 1.0, "改装済": 0.0})

    # 用途
    col = "用途"
    d = {}
    for val in df[col].value_counts().keys():
        if "、" in val:
            d[val] = val.split("、")[0]
    df[col] = df[col].replace(d)

    # 建築年
    col = "建築年"
    d = {}
    for val in df[col].value_counts().keys():
        if "平成" in val:
            _year = int(val.split("平成")[1].split("年")[0])
            year = 1988 + _year
        if "昭和" in val:
            _year = int(val.split("昭和")[1].split("年")[0])
            year = 1925 + _year
        if "令和" in val:
            _year = int(val.split("令和")[1].split("年")[0])
            year = 2018 + _year
        d[val] = year
    d["戦前"] = 1945
    df[col] = df[col].replace(d)

    # col = "間取り"
    col = "間取り"
    ints = ["１", "２", "３", "４", "５", "６", "７", "８", "９"]

    d1 = {}
    dr = defaultdict(lambda: defaultdict(int))
    for k in sorted(df[col].value_counts().keys()):
        if k[0] in ints:
            d1[k] = int(k[0])
        else:
            d1[k] = 0

        for room_type in ["Ｌ", "Ｄ", "Ｋ", "Ｓ"]:
            dr[room_type][k] = k.count(room_type)

    df[f"{col}_otherroom"] = df[col].replace(d1)
    df[f"{col}_L"] = df[col].replace(dr["Ｌ"])
    df[f"{col}_D"] = df[col].replace(dr["Ｄ"])
    df[f"{col}_K"] = df[col].replace(dr["Ｋ"])
    df[f"{col}_S"] = df[col].replace(dr["Ｓ"])
    df.drop(col, axis=1, inplace=True)

    # 面積
    col = "面積（㎡）"
    if flag == "train":
        df[col] = df[col].replace({"2000㎡以上": 2000})
    df[col] = df[col].astype(int)

    # 距離
    col = "最寄駅：距離（分）"
    d = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "2H?": 120,
        "1H30?2H": 105,
    }
    df[col] = df[col].replace(d)
    df[col] = df[col].astype(float)

    for col in ["用途", "地区名", "最寄駅：名称", "建物の構造", "今後の利用目的", "都市計画"]:
        df[col] = df[col].astype("category")
    
    return df

In [32]:
# 前処理
df_train = preprocess(df_train, flag="train")
df_test = preprocess(df_test, flag="test")

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

In [33]:
clf = lgb.LGBMRegressor(random_state=0)
clf.fit(train_x, train_y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
vals = clf.predict(df_test)

In [35]:
df_test["取引価格（総額）_log"] = vals

In [37]:
df_test[["取引価格（総額）_log"]].to_csv("submission.csv")

## 今後について

とりあえず、機械学習を回すように変換はできました。（現状Scoreが0.0881）

現状LightGBMのパラメーターがデフォルトだったり、カテゴリカル変数の処理などもほぼ無処理なので、このあたりの前処理を頑張ることでスコアをあげていけそうです。
